In [63]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint
from bs4 import BeautifulSoup
import time
import re

In [2]:
#creating a header file to 
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}   

In [3]:
# Write a function to make a call to the yelp API

In [37]:
from keys import client_id,api_key
url =  'https://api.yelp.com/v3/businesses/search'
term='Restaurants'
location='Brooklyn'

url_params={
    'term': term.replace(' ','+'),
    'location': location.replace(' ','+'),
    'limit': 50
}



In [18]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
    data= requests.get(url, headers=headers, params=url_params)
    return data

In [19]:
response = yelp_call(url_params, api_key)
response.status_code

200

In [7]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
response.json()['total']

22500

In [3]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB


In [48]:
data=response.json()['businesses']
data[0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'location', 'phone', 'display_phone', 'distance'])

In [26]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    # create a container to hold our parsed data
    parsed_data = []
    businesses = results.json()['businesses']
    # loop through our business and parse each individual business
    for business in businesses:
        #print(business['name'])
        if 'price' in list(business.keys()):
            biz_list = [
                business['id'],
                business['name'], 
                business['url'],
                business['review_count'],
                [x['alias'] for x in business['categories']],
                business['rating'],
                business['price']
            ]
        else:
            biz_list = [
                business['id'],
                business['name'], 
                business['url'],
                business['review_count'],
                [x['alias'] for x in business['categories']],
                business['rating'],
                np.nan
            ]
    # in the loop, add each individual parsed  business to our data container
        parsed_data.append(biz_list)
    return parsed_data

In [27]:
parsed_data=parse_results(response)

In [28]:
parsed_data

[['V7lXZKBDzScDeGB8JmnzSA',
  "Katz's Delicatessen",
  'https://www.yelp.com/biz/katzs-delicatessen-new-york?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
  12280,
  ['delis', 'sandwiches', 'catering'],
  4.0,
  '$$'],
 ['44SY464xDHbvOcjDzRbKkQ',
  'Ippudo NY',
  'https://www.yelp.com/biz/ippudo-ny-new-york-7?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
  9970,
  ['ramen'],
  4.0,
  '$$'],
 ['xEnNFXtMLDF5kZDxfaCJgA',
  'The Halal Guys',
  'https://www.yelp.com/biz/the-halal-guys-new-york-2?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
  9457,
  ['foodstands', 'mideastern', 'halal'],
  4.0,
  '$'],
 ['0CjK3esfpFcxIopebzjFxA',
  "Joe's Shanghai",
  'https://www.yelp.com/biz/joes-shanghai-new-york-2?adjust_creative=iiD4cWOR-Fz3f

In [1]:
# Write a function to take your parsed data and insert it into CSV

In [29]:
def data_save(parsed_results, csv_filename):
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.
    cols=['ID','Name','url','Reviews','categories','Rating','Price']
    df=pd.DataFrame(parsed_results,columns=cols)
    df.to_csv(csv_filename)

In [38]:
# Write a script that combines the three functions above into a single process.

# Keep all results in a list to be made into a df later
ready_data=[]
# List of locations to search in
boroughs=['Queens','Brooklyn','Bronx','Manhattan','Staten+Island']

for borough in boroughs:
    url_params['location'] = borough
# create a variable  to keep track of which result you are in. 
    cur = 0

    #set up a while loop to go through and grab the result 
    while  cur < 1000:
        #set the offset parameter to be where you currently are in the results 
        url_params['offset'] = cur
        #make your API call with the new offset number
        results = yelp_call(url_params, api_key)

        #after you get your results you can now use your function to parse those results
        parsed_results = parse_results(results)

        # add to list of results
        ready_data.extend(parsed_results)
        #increment the counter by 50 to move on to the next results
        cur += 50

In [39]:
len(ready_data)

5000

In [40]:
data_save(ready_data, 'yelp_data_restaurant.csv')

In [121]:
df=pd.read_csv('yelp_data_restaurant.csv',index_col=0)

df.head(14)

,ID,Name,url,Reviews,categories,Rating,Price
0,hWmzrNyhPO3phBstCynSvw,Rainhas Churrascaria,https://www.yelp.com/biz/rainhas-churrascaria-...,734,"['brazilian', 'steak']",4.0,$$$
1,HiTAdo-v0S1nSfkXfynnHg,Lim's kitchen,https://www.yelp.com/biz/lims-kitchen-long-isl...,91,"['chicken_wings', 'korean']",5.0,NaN
2,o_r0-1tGDoP_HyEONWKqCw,Mojo Restaurant,https://www.yelp.com/biz/mojo-restaurant-fores...,236,['latin'],4.0,$$
3,3GMHPnrkFAcqffw3T7JSdQ,Kitchen 79,https://www.yelp.com/biz/kitchen-79-jackson-he...,615,['thai'],4.5,$$
4,kBZggrnSP1kcUMnsnfkTaQ,Nan Xiang Xiao Long Bao,https://www.yelp.com/biz/nan-xiang-xiao-long-b...,3555,"['shanghainese', 'gourmet', 'noodles']",4.0,$
5,0L03onqNwxWLrbb_FCK79A,SriPraPhai Thai Restaurant,https://www.yelp.com/biz/sripraphai-thai-resta...,2330,['thai'],4.0,$$
6,zekbMn7KWtzxTFLtfEqb1w,Rincon Criollo,https://www.yelp.com/biz/rincon-criollo-corona...,799,['cuban'],4.5,$$
7,19o6iblcJvz2hdPouYhXgg,Zum Stammtisch,https://www.yelp.com/biz/zum-stammtisch-glenda...,462,['german'],4.0,$$
8,hKtSEqh8fLgLneDdZZwF8w,Kusina Pinoy Bistro,https://www.yelp.com/biz/kusina-pinoy-bistro-w...,94,['filipino'],4.5,$$
9,nkfUKb7oWQt54Uz9VQLhUg,Butcher Bar,https://www.yelp.com/biz/butcher-bar-astoria-2...,2008,"['bbq', 'burgers']",4.0,$$


In [46]:
df['url'].values

array(['https://www.yelp.com/biz/rainhas-churrascaria-corona?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
       'https://www.yelp.com/biz/lims-kitchen-long-island-city-2?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
       'https://www.yelp.com/biz/mojo-restaurant-forest-hills?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
       ...,
       'https://www.yelp.com/biz/thursday-kitchen-new-york?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
       'https://www.yelp.com/biz/the-old-spot-jersey-city-5?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
       'htt

In [52]:
df.drop_duplicates(inplace=True)
df[df['Reviews']>50].count()

ID            3300
Name          3300
url           3300
Reviews       3300
categories    3300
Rating        3300
Price         3195
dtype: int64

In [96]:
url_random = 'https://www.yelp.com/biz/rainhas-churrascaria-corona?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg&start=20'
response_soup_random = requests.get(url_random, headers = header)
soup_random = BeautifulSoup(response_soup_random.text, 'html.parser')

In [64]:
re.sub('&start=.*$','',url_random,flags=re.DOTALL)

'https://www.yelp.com/biz/rainhas-churrascaria-corona?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg'

In [97]:
revs=soup_random.find_all('li', {'class': 'lemon--li__373c0__1r9wz margin-b3__373c0__q1DuY padding-b3__373c0__342DA border--bottom__373c0__3qNtD border-color--default__373c0__3-ifU'})
revs[1].find('span', {'class': 'lemon--span__373c0__3997G raw__373c0__3rKqk'}).text

"Excellent restaurant for meat lovers. We went here on a Saturday night no need for reservation, we were seated immediately. Buffet cost 45 per person and it was worth it. Variety of dishes available including salads, rice. We didn't even have to go back to the Buffet area because different types of meat dishes were being served. The servers go around the table serving the meat dishes. Drinks have separate prices which was not bad. Had a lot of fun! Servers were very friendly and attentive to our needs. Definitely we will come back there."

In [107]:
# create an empty list of show_reviews
restaurant_review_list = []

# function to get each review, reviewer and score and save it in a dictionary along with the
# title of the game
def each_review(url):
    
    time.sleep(2)
    
# getting the beautiful soup parser initialized, changes as the url changes which will be 
# looped in the next block
    response_soup = requests.get(url, headers = header)
    soup = BeautifulSoup(response_soup.text, 'html.parser')
    
# try and except for each page to make sure the page exists and it does not go straight to
# a 404 error page, if it does then it will skip that page and move onto the next one
    try:


# this grabs the entire block of reviews on the page, used to loop through to grab individual
# reviews and scores
        reviews_list = soup.find_all('li', {'class': 'lemon--li__373c0__1r9wz margin-b3__373c0__q1DuY padding-b3__373c0__342DA border--bottom__373c0__3qNtD border-color--default__373c0__3-ifU'})
    except AttributeError:
            pass
    
    try:
        
#looping through the reviews in the block to get the user, review and score
        for x in reviews_list:
            try:
                #user = x.find('div', {'style': 'float: left;'})('a')[1].text
                #user_score = int((x.find('div', {'class': 'mb8'})('div')[2].text).split()[-1])
                user_review = x.find('span', {'class': 'lemon--span__373c0__3997G raw__373c0__3rKqk'}).text

# Storing the title, user, user score and review in each iteration in a dictionary that is 
# overwritten at every new iteration
                restaurant_review_dict = {
                    'url': re.sub('&start=.*$','',url,flags=re.DOTALL),
                    'user_review' : user_review
                }

# stores the dictionary in a list each iteration                
                restaurant_review_list.append(restaurant_review_dict)
            except AttributeError:
                pass
            
    except UnboundLocalError:
        pass
    
    
    return restaurant_review_list

In [66]:
df.url.values[0]

'https://www.yelp.com/biz/rainhas-churrascaria-corona?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg'

In [129]:
df[df['url']=='https://www.yelp.com/biz/o-lavrador-jamaica-2?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg']#['Reviews'].values[0]

,ID,Name,url,Reviews,categories,Rating,Price
97,d61Ejg_qQGTATqh67c912g,O Lavrador,https://www.yelp.com/biz/o-lavrador-jamaica-2?...,240,"['portuguese', 'bars', 'spanish']",4.0,$$


In [126]:
df[df['url']=='https://www.yelp.com/biz/kaia-wine-bar-new-york?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg']#['Reviews'].values[0]

,ID,Name,url,Reviews,categories,Rating,Price
3408,6tGW6KMFB1rLGInE6sB3KA,Kaia Wine Bar,https://www.yelp.com/biz/kaia-wine-bar-new-yor...,422,"['southafrican', 'tapas', 'cocktailbars']",4.0,$$


In [104]:
c=0
while c < (df[df['url']==df.url.values[0]]['Reviews'].values[0]) and c<300:
    url = df.url.values[0] + '&start={}'.format(c)
    each_review(url)
    c+=20

In [125]:
# loop through the function constantly changing the url using the list of hyperlinks
# created earlier

for link in df.url.values[98:]:
    c=0
    
    while c < (df[df['url']==link]['Reviews'].values[0]) and c<50:
        url = link + '&start={}'.format(c)
        if (len(restaurant_review_list)//3000)%2==1:
            time.sleep(1)
        each_review(url)
        c+=20

ConnectionError: HTTPSConnectionPool(host='www.yelp.com', port=443): Max retries exceeded with url: /biz/kaia-wine-bar-new-york?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg&start=60 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000022F5CE100C8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [128]:
restaurant_review_list[11238]

{'url': 'https://www.yelp.com/biz/o-lavrador-jamaica-2?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=iiD4cWOR-Fz3fX4le1knTg',
 'user_review': 'Last night we were celebrating 2 family birthday ladies and one belated Virgo birthday, 5 of us all total. I had a Groupon coupon for 4 dinners, wine, appetizers, and entrees. \xa0We paid a small fee upgrade for some of the dishes, and it was money well spent! Eloisa was such a wonderful Hostess, and our waiter took excellent care of us!The food was fresh, delicious, and so nicely presented. There is a parking lot, and this is a lovely little gem not far from our home. Everthing we ordered from the menu was just soooo good. Definitly have the seafood paella, the baked clams with cheese, and the little seafood empanadas. \xa0There was a guitarist playing softly in the background which softly accented our dinner...... Thank you for a lovely evening !'}

In [132]:
len(restaurant_review_list)

11239

In [130]:
# store the list of dictionaries into a dataframe

df_temp = pd.DataFrame(restaurant_review_list)

# store the dataframe as a csv file

df_temp.to_csv('restaurant_reviews02.csv')

In [143]:
import json

all_businesses=[]
for business in open('../yelp_dataset/yelp_academic_dataset_business.json','rb'):
    all_businesses.append(json.loads(business))

In [144]:
df_business=pd.DataFrame(all_businesses)

In [189]:
df_business.head(30)

,ID,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",0
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None,0
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None,0
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",0
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",0
5,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ...",0
6,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,"6870 S Rainbow Blvd, Ste 117",Las Vegas,NV,89118,36.063977,-115.241463,3.5,81,1,"{'BusinessParking': '{'garage': False, 'street...","Dry Cleaning & Laundry, Local Services, Laundr...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ...",0
7,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,6910 E Southern Ave,Mesa,AZ,85209,33.393885,-111.682226,5.0,18,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Auto Repair, Oil Change Stations, Automotive, ...","{'Monday': '7:30-17:0', 'Tuesday': '7:30-17:0'...",0
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1
9,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,700 Kipling Avenue Etobicoke,Toronto,ON,M8Z 5G3,43.624539,-79.529108,3.0,16,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Martial Arts, Gyms, Fitness & Instruction, Act...","{'Monday': '5:30-23:0', 'Tuesday': '5:30-23:0'...",0


In [187]:
df_business.categories.values[25]

'Japanese, Fast Food, Food Court, Restaurants'

In [188]:
df_business['Restaurants']=df_business.categories.apply(lambda x: 1 if 'Restaurants' in str(x).split(', ') else 0)

In [184]:
df_business[df_business.Restaurants==1]

,ID,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,1,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Cheesesteaks, Poutineries","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1
38,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","Restaurants, Burgers, Food",None,1
41,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,"1541 E 38th St, Ste 101",Cleveland,OH,44114,41.512155,-81.663332,4.5,23,1,"{'BusinessParking': 'None', 'Ambience': '{'tou...","Restaurants, Vietnamese, Soup","{'Monday': '11:0-20:0', 'Wednesday': '11:0-20:...",1
43,LoRef3ChgZKbxUio-sHgQg,Amir,5252 Rue Jean Talon O,Montréal,QC,H4P 2A7,45.494870,-73.651904,3.0,18,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Lebanese, Middle Eastern",None,1
49,tLpkSwdtqqoXwU0JAGnApw,Wendy's,4602 Northfield Road,Cleveland,OH,44128,41.434614,-81.527026,3.5,7,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Fast Food, Burgers","{'Monday': '10:0-3:0', 'Tuesday': '10:0-3:0', ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209343,8dK-8e006F_it5bNxTVlaQ,Famous Dave's Bar-B-Que,26410 Great Northern Shopping Ctr,North Olmsted,OH,44070,41.418407,-81.915597,3.5,120,1,"{'Caters': 'True', 'BusinessAcceptsCreditCards...","Restaurants, Barbeque, Comfort Food, Salad, Ev...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-17:0', ...",1
209359,GAgEoHcf4PSuZRS5Zd3ltA,Q's Shawarma,1075 Martin Grove Road,Toronto,ON,M9W 4W6,43.701807,-79.575135,4.0,16,1,"{'GoodForKids': 'True', 'HasTV': 'True', 'Rest...","Restaurants, Mediterranean","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",1
209372,wjZ12shpUWAy4Boy0G5z1g,Island Buffet,"Tropicana Resort and Casino, 3801 Las Vegas Bl...",Las Vegas,NV,89109,36.099378,-115.171029,1.5,9,0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, Buffets",None,1
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,0,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...","Restaurants, Sandwiches, Food, Breakfast & Bru...","{'Monday': '5:30-15:0', 'Tuesday': '5:30-15:0'...",1


In [160]:
df_business.shape

(209393, 15)

In [134]:
import json

all_reviews=[]
for review in open('../yelp_dataset/yelp_academic_dataset_review.json','rb'):
    all_reviews.append(json.loads(review))

In [135]:
df_review=pd.DataFrame(all_reviews)

In [139]:
df_review.head()

,review_id,user_id,ID,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [193]:
df_business[df_business.business_id=='HQl28KMwrEKHqhFrrDqVNQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants
18787,HQl28KMwrEKHqhFrrDqVNQ,Deagan's Kitchen & Bar,14810 Detroit Ave,Lakewood,OH,44107,41.485192,-81.800145,4.0,417,1,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Restaurants, Gastropubs, Nightlife, American (...","{'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22...",1
